In [1]:
import sys
import time
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../models/STimage/stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling, ensembl_to_id
from stimage._model import CNN_NB_multiple_genes, negative_binomial_layer, negative_binomial_loss
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
# import geopandas as gpd
from sklearn.neighbors import KDTree
from anndata import read_h5ad
from tensorflow.keras import backend as K
import scanpy as sc

import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation
import pickle
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r


DATA_PATH = Path("../data/pfizer")
OUT_PATH = Path("../trained_models")

adata_all = read_h5ad(DATA_PATH / "all_adata.h5ad")
# adata_all.obs['tile_path'] = adata_all.obs['tile_path'].apply(lambda x: x.replace('/clusterdata/uqxtan9/Q1851/Xiao/Working_project/','../data/'))

# adata_all = ensembl_to_id(adata_all)

samples = adata_all.obs["library_id"].unique().tolist()

# gene_list=["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", 
#            "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

# gene_list_path = "./gene_list.pkl"
# with open(gene_list_path, 'rb') as f:
#     gene_list = pickle.load(f)

from read_stimage_genes import read_gene_set_hvg
gene_list = read_gene_set_hvg("../data/pfizer/", out="list")

df = pd.DataFrame()

i = int(sys.argv[1])
test_sample = samples[i]
n_genes = len(gene_list)
print("test sample", test_sample)

print("number of genes: ", n_genes)

adata_all_train_valid = adata_all[adata_all.obs["library_id"].isin(
    adata_all.obs.library_id.cat.remove_categories(test_sample).unique())]

training_index = adata_all_train_valid.obs.sample(frac=0.7, random_state=1).index
training_dataset = adata_all_train_valid[training_index,].copy()

valid_index = adata_all_train_valid.obs.index.isin(training_index)
valid_dataset = adata_all_train_valid[~valid_index,].copy()

test_index = adata_all.obs.library_id == test_sample
test_dataset_1 = adata_all[test_index,].copy()

print("num obs")
print(test_dataset_1.num_obs)


ModuleNotFoundError: No module named 'stlearn'

In [5]:
from anndata import read_h5ad
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import sys
import time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../models/STimage/stimage").resolve()


In [7]:
DATA_PATH = Path("../data/pfizer")
OUT_PATH = Path("../trained_models")

adata_all = read_h5ad(DATA_PATH / "all_adata.h5ad")
# adata_all.obs['tile_path'] = adata_all.obs['tile_path'].apply(lambda x: x.replace('/clusterdata/uqxtan9/Q1851/Xiao/Working_project/','../data/'))

# adata_all = ensembl_to_id(adata_all)

samples = adata_all.obs["library_id"].unique().tolist()

# gene_list=["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", 
#            "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

# gene_list_path = "./gene_list.pkl"
# with open(gene_list_path, 'rb') as f:
#     gene_list = pickle.load(f)

from read_stimage_genes import read_gene_set_hvg
gene_list = read_gene_set_hvg("../data/pfizer/", out="list")

df = pd.DataFrame()

i = 1
test_sample = samples[i]
n_genes = len(gene_list)
print("test sample", test_sample)

print("number of genes: ", n_genes)

adata_all_train_valid = adata_all[adata_all.obs["library_id"].isin(
    adata_all.obs.library_id.cat.remove_categories(test_sample).unique())]

training_index = adata_all_train_valid.obs.sample(frac=0.7, random_state=1).index
training_dataset = adata_all_train_valid[training_index,].copy()

valid_index = adata_all_train_valid.obs.index.isin(training_index)
valid_dataset = adata_all_train_valid[~valid_index,].copy()

test_index = adata_all.obs.library_id == test_sample
test_dataset_1 = adata_all[test_index,].copy()


test sample VLP79_D
number of genes:  1000


In [11]:
adata_all_train_valid

View of AnnData object with n_obs × n_vars = 35954 × 18085
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow', 'tile_tissue_mask_path', 'tissue_area', 'tile_path', 'library_id'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [9]:
adata_all_train_valid.obs['library_id']

AACACTTGGCAAGGAA-1-VLP79_A    VLP79_A
AACAGGATTCATAGTT-1-VLP79_A    VLP79_A
AACAGGTTCACCGAAG-1-VLP79_A    VLP79_A
AACAGTCCACGCGGTG-1-VLP79_A    VLP79_A
AACATAGTCTATCTAC-1-VLP79_A    VLP79_A
                               ...   
TGTTGGAACGAGGTCA-1-VLP78_A    VLP78_A
TGTTGGAAGCTCGGTA-1-VLP78_A    VLP78_A
TGTTGGATGGACTTCT-1-VLP78_A    VLP78_A
TGTTGGCCAGACCTAC-1-VLP78_A    VLP78_A
TGTTGGCCTACACGTG-1-VLP78_A    VLP78_A
Name: library_id, Length: 35954, dtype: category
Categories (9, object): ['VLP79_A', 'VLP80_A', 'VLP80_D', 'VLP81_A', ..., 'VLP82_D', 'VLP83_A', 'VLP83_D', 'VLP78_A']

In [15]:
training_dataset

AnnData object with n_obs × n_vars = 25168 × 18085
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow', 'tile_tissue_mask_path', 'tissue_area', 'tile_path', 'library_id'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [16]:
valid_dataset

AnnData object with n_obs × n_vars = 10786 × 18085
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow', 'tile_tissue_mask_path', 'tissue_area', 'tile_path', 'library_id'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [14]:
test_dataset_1.obs['library_id']

AACACGTGCATCGCAC-1-VLP79_D    VLP79_D
AACAGGAAGAGCATAG-1-VLP79_D    VLP79_D
AACAGGATTCATAGTT-1-VLP79_D    VLP79_D
AACAGGTTATTGCACC-1-VLP79_D    VLP79_D
AACAGGTTCACCGAAG-1-VLP79_D    VLP79_D
                               ...   
TGTTGGAACCTTCCGC-1-VLP79_D    VLP79_D
TGTTGGAACGAGGTCA-1-VLP79_D    VLP79_D
TGTTGGATGGACTTCT-1-VLP79_D    VLP79_D
TGTTGGCCAGACCTAC-1-VLP79_D    VLP79_D
TGTTGGCCTACACGTG-1-VLP79_D    VLP79_D
Name: library_id, Length: 3135, dtype: category
Categories (1, object): ['VLP79_D']